In [1]:
import glob
import pandas as pd
import datetime
import calendar
import numpy as np

In [20]:
# takes around 3 minutes to run

# get report names
reportnames = glob.glob('JSC370 Data KM/*.*', recursive = True)

# read in all excel files
report_data = pd.DataFrame()
for name in reportnames:
    if "JR1" not in name:
        print("Not JR1")
        print(name)
        continue
    
    excel = True
    try:
        curr_report = pd.read_excel(name)
    except:
        try:
            curr_report = pd.read_csv(name, skiprows=7, sep="\t")
        except:
            curr_report = pd.read_csv(name, skiprows=9, sep="\t")
        excel = False
    
    if excel:
        # get index where data starts 
        # (this is different between some files so using a loop to get the starting point)
        colnamesindex = -1
        for i in range(len(curr_report)):
            if curr_report.iloc[i, 0] == "Journal" or curr_report.iloc[i, 1] == "Publisher":
                colnamesindex = i
                break
        if colnamesindex == -1:
            print("Might be in a different language, not matching up properly")
            print(name)
            continue

        # set column names as the proper thing
        curr_report.columns = curr_report.loc[colnamesindex,]

        if "+" in curr_report.columns:
            # for the files formated weirdly with a + and spreading title names across rows
            # fix column names
            colnames = list(curr_report.columns)
            for i in range(len(colnames)):
                if pd.notna(curr_report.iloc[colnamesindex + 1, i]):
                    colnames[i] = colnames[i] + " " + str(curr_report.iloc[colnamesindex + 1, i])
                if pd.notna(curr_report.iloc[colnamesindex + 2, i]):
                    colnames[i] = colnames[i] + " " + str(curr_report.iloc[colnamesindex + 2, i])
            curr_report.columns = colnames
            curr_report = curr_report.drop(columns=["+"])
            curr_report = curr_report.drop(curr_report[curr_report["Journal"] == "+"].index, axis=0)

            # save only data part
            curr_report = curr_report[colnamesindex + 3:]
        else:
            # save only data part
            curr_report = curr_report[colnamesindex + 1:]

    # insert year of report
    jr1_i = name.index("JR1")
    year = name[jr1_i + 4 : jr1_i + 8]
    curr_report.insert(0, "Year", int(year))

    # reformat months to not include year or date for generality
    datetimes = {}
    for colname in curr_report.columns:
        if pd.notna(colname):
            if isinstance(colname, datetime.datetime):
                # some encoded as datetime
                calendar.month_name[colname.month][:3]
                datetimes[colname] = calendar.month_name[colname.month][:3]
            elif isinstance(colname, str):
                # sometimes there's whitespace messing things up
                strippedcolname = colname.strip()
                if strippedcolname.endswith(year):
                    # some encoded as MMM-YYYY
                    datetimes[colname] = strippedcolname[:3]
                else:
                    datetimes[colname] = strippedcolname
    curr_report = curr_report.rename(columns=datetimes)

    # for some reports that don't label the Journal column
    curr_report = curr_report.rename(columns={np.nan: "Journal"})
    
    # insert file package name (not sure if useful)
    package_i = name.index("\\")
    filepub = name[package_i + 1 : jr1_i - 1]
    curr_report.insert(0, "FilePackage", filepub)
    
    # drop totals row
    curr_report = curr_report.drop(curr_report.index[0])
    
    # insert if SP or not
    mainname = name.split(".")[0]
    if mainname.endswith("SP"):
        curr_report.insert(0, "SP", "Yes")
    else:
        curr_report.insert(0, "SP", "No")
    
    try:
        report_data = report_data.append(curr_report)
    except ValueError:
        print("Can't append, some other weird error")
        print(name)
        continue
    
print("Done")

Not JR1
JSC370 Data KM\ALJC JR5 2017 SP.xlsx
Not JR1
JSC370 Data KM\ALJC JR5 2018 SP.xlsx
Might be in a different language, not matching up properly
JSC370 Data KM\CAIRN JR1 2014.xls
Might be in a different language, not matching up properly
JSC370 Data KM\CAIRN JR1 2015.xls
Might be in a different language, not matching up properly
JSC370 Data KM\CAIRN JR1 2016 .xls
Might be in a different language, not matching up properly
JSC370 Data KM\CAIRN JR1 2017.xls
Might be in a different language, not matching up properly
JSC370 Data KM\CAIRN JR1 2018.xls
Done


In [21]:
# fixing more formatting problems
report_data["Reporting Period Total"] = report_data["Reporting Period Total"].fillna(report_data["Retrievals"])
report_data["Reporting Period HTML"] = report_data["Reporting Period HTML"].fillna(report_data["HTML"])
report_data["Reporting Period PDF"] = report_data["Reporting Period PDF"].fillna(report_data["PDF"])
report_data["Journal DOI"] = report_data["Journal DOI"].fillna(report_data["Journal Doi"])
report_data["Journal"] = report_data["Journal"].fillna(report_data["Title"])
report_data["Journal"] = report_data["Journal"].fillna(report_data["Unnamed: 0"])

report_data = report_data.drop(columns=["Retrievals", "HTML", "PDF", "Journal Doi", "Title", "Dec-2015", "Unnamed: 0"])

report_data.loc[report_data["Online ISSN"] == " ", "Online ISSN"] = np.nan

# drop the rows that are accidentally still there
report_data = report_data[report_data["Reporting Period Total"].notna()]

In [29]:
# strip excess whitespace
report_data["Journal"] = report_data["Journal"].apply(lambda x: x.strip() if isinstance(x, str) else x)
report_data["FilePackage"] = report_data["FilePackage"].apply(lambda x: x.strip() if isinstance(x, str) else x)

# double check data
report_data["Reporting Period Total"] = report_data["Reporting Period Total"].apply(
    lambda x: x if isinstance(x, int) else np.nan)
report_data = report_data.dropna(subset=["Reporting Period Total"])

In [32]:
report_data.head()

,SP,FilePackage,Year,Journal,Publisher,Platform,Journal DOI,Proprietary Identifier,Print ISSN,Online ISSN,...,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,No,AAAS,2014,Science,American Association for the Advancement of Sc...,HighWire Press,NaN,NaN,0036-8075,1095-9203,...,11465,8414,7123,6212,6373,5904,8603,11097,10606,6989
1,No,AAAS,2014,Science Advances,American Association for the Advancement of Sc...,HighWire Press,NaN,NaN,NaN,2375-2548,...,0,0,0,0,0,0,0,0,0,0
2,No,AAAS,2014,Science Signaling,American Association for the Advancement of Sc...,HighWire Press,NaN,NaN,1945-0877,1937-9145,...,514,333,239,277,293,211,315,387,334,304
3,No,AAAS,2014,Science Translational Medicine,American Association for the Advancement of Sc...,HighWire Press,NaN,NaN,1946-6234,1946-6242,...,855,729,545,523,472,478,701,932,664,431
4,No,AAAS,2014,Science,American Association for the Advancement of Sc...,HighWire Press,NaN,NaN,0036-8075,1095-9203,...,11465,8414,7123,6212,6373,5904,8603,11097,10606,6989


In [25]:
# uoft report names
uoftnames = glob.glob('Web of Science data UofT affiliated pubs 2014-2018/*.*', recursive = True)

# takes around 40 seconds to load
uoft_data = pd.DataFrame()
year = 2014
for name in uoftnames:
    curr_uoft = pd.read_excel(name)
    curr_uoft.insert(0, "Year", year)
    uoft_data = uoft_data.append(curr_uoft)
    year = year + 1
    
uoft_data = uoft_data.drop_duplicates()
uoft_data = uoft_data.reset_index(drop=True)

In [26]:
# fix formatting problems
uoft_data["Category: Heading 1"] = uoft_data["Category: Heading 1"].fillna(uoft_data["Category: Headings 1"])
uoft_data["PubType"] = uoft_data["PubType"].fillna(uoft_data["Pubtype"])

uoft_data = uoft_data.drop(columns=["Category: Headings 1", "Pubtype"])

In [27]:
# TODO: strip column names sometime

# Add number of uoft authors to each entry
# takes 2:30 minutes to run
first_three = ["(a1) First UofT affiliated author's position in the author list ",
               " (a2) Second UofT affiliated author's position in the author list",
               "(a3) Third UofT affiliated author's position in the author list "]

uoft_data["NumUofTAuthors"] = 0
for i, r in uoft_data.iterrows():
    if pd.notna(uoft_data.loc[i, first_three[0]]):
        uoft_data.loc[i, "NumUofTAuthors"] = uoft_data.loc[i, "NumUofTAuthors"] + 1
    else:
        continue
        
    if pd.notna(uoft_data.loc[i, first_three[1]]):
        uoft_data.loc[i, "NumUofTAuthors"] = uoft_data.loc[i, "NumUofTAuthors"] + 1
    else:
        continue
        
    if pd.notna(uoft_data.loc[i, first_three[2]]):
        uoft_data.loc[i, "NumUofTAuthors"] = uoft_data.loc[i, "NumUofTAuthors"] + 1
    else:
        continue
        
    for j in range(4, 61):
        if pd.notna(uoft_data.loc[i, 'a' + str(j)]):
            uoft_data.loc[i, "NumUofTAuthors"] = uoft_data.loc[i, "NumUofTAuthors"] + 1
        else:
            break
            
print("Done")

Done


In [33]:
uoft_data.head()

,Year,UID,PubDate,Issue,Volume,Pages,Start Page,End Page,Number of Pages,Source Title,...,b56,a57,b57,a58,b58,a59,b59,a60,b60,NumUofTAuthors
0,2014,WOS:000341974900031,2014-08-01,2,98,541-548,541,548,9,ANNALS OF THORACIC SURGERY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2014,WOS:000346385100013,2014-01-01,11,138,1495-1502,1495,1502,8,ARCHIVES OF PATHOLOGY & LABORATORY MEDICINE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2014,WOS:000342917300011,2014-08-01,4,23,302-307,302,307,6,CURRENT DIRECTIONS IN PSYCHOLOGICAL SCIENCE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,2014,WOS:000331684800009,2014-03-01,3,66,404-410,404,410,7,ARTHRITIS CARE & RESEARCH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2014,WOS:000331927800004,2014-03-01,3,71,916-928,916,928,13,JOURNAL OF THE ATMOSPHERIC SCIENCES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


# User Selecting Data Subgroup

In [34]:
# set by user

# month breakdown can only happen if type of downloads is total
# selecting other things like subgroup of publishers or journal names is easy panda methods

type_of_downloads = "total" # ["total", "pdf", "html"] 
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
start_year = 2014
start_month = "Jan"
end_year = 2018
end_month = "Dec"
######

In [35]:
# will create SettingWithCopyWarning even though it's fine
def get_selected_report_data(type_of_downloads, start_year, start_month, end_year, end_month):
    selected_data = pd.DataFrame()
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    years = []
    for i in range(end_year - start_year + 1):
        years.append(start_year + i)

    d_type = ""
    if type_of_downloads == "total":
        d_type = "Reporting Period Total"
    elif type_of_downloads == "pdf":
        d_type = "Reporting Period PDF"
    elif type_of_downloads == "html":
        d_type = "Reporting Period HTML"
    else:
        # download type not valid
        print("Download Type not valid")

    years = []
    for i in range(end_year - start_year + 1):
        years.append(start_year + i)

    # monthly specifics only available if download type is total
    if type_of_downloads == "total":
        start_month_i = months.index(start_month)
        end_month_i = months.index(end_month)
        if start_year == end_year:
            # only within the year range
            selected_data = report_data[report_data["Year"] == start_year]
            selected_data["Downloads"] = 0

            for j in range(start_month_i, end_month_i + 1):
                selected_data.loc[:, "Downloads"] = selected_data.loc[:, "Downloads"] + selected_data[months[j]].fillna(0)
        
        elif start_year < end_year:
            # start year months
            selected_data = report_data[report_data["Year"] == start_year]
            selected_data["Downloads"] = 0

            # get counts within month range for start year
            for j in range(start_month_i, len(months)):
                selected_data.loc[:, "Downloads"] = selected_data.loc[:, "Downloads"] + selected_data[months[j]].fillna(0)

            # inbetween years entire year
            inbetween_years = years[1 : len(years) - 1]
            inbetween_data = report_data[report_data["Year"].isin(inbetween_years)]
            inbetween_data.loc[:, "Downloads"] = inbetween_data.loc[:, d_type]

            # loop for end year months
            end_data = report_data[report_data["Year"] == end_year]
            end_data["Downloads"] = 0

            for j in range(0, end_month_i + 1):
                end_data.loc[:, "Downloads"] = end_data.loc[:, "Downloads"] + end_data[months[j]].fillna(0)

            selected_data = selected_data.append(inbetween_data)
            selected_data = selected_data.append(end_data)

        else:
            # start year > end year, no data
            print("Start year after end year, no data")
            selected_data = pd.DataFrame()
    else:
        # do only year selections
        selected_data = report_data[report_data["Year"].isin(years)]
        selected_data.loc[:, "Downloads"] = report_data[d_type]
    
    return selected_data

In [36]:
# this merging method probably needs work, to take into account missing ISSNs in some rows and stuff
def merge_selected_reports_and_uoft(selected_data):
    # ISSNs of only selected data
    available_links = selected_data[["Print ISSN", "Online ISSN"]]
    available_links = available_links.drop_duplicates()
    available_links = available_links[(available_links["Print ISSN"].notna()) | 
                                      (available_links["Online ISSN"].notna())]
    available_links = available_links.reset_index(drop=True)

    # get uoft publications that match the selected ISSNs
    matched_uoft = pd.merge(uoft_data, available_links, how="inner", 
                            left_on=["ISSN", "eISSN"], right_on=["Print ISSN", "Online ISSN"])

    # save their ISSNs
    matched_uoft = matched_uoft.drop(columns=["ISSN", "eISSN"])
    available_links = matched_uoft[["Print ISSN", "Online ISSN"]]
    available_links = available_links.drop_duplicates()
    available_links = available_links[(available_links["Print ISSN"].notna()) | 
                                      (available_links["Online ISSN"].notna())]
    available_links = available_links.reset_index(drop=True)

    # get download counts of those uoft publications
    matched_selected = pd.merge(selected_data, available_links, how="inner", 
                            on=["Print ISSN", "Online ISSN"])

    # takes a while and can cause memory problems if both dataframes really big
    matched_data = pd.merge(matched_selected, matched_uoft, how="inner", 
                            on=["Print ISSN", "Online ISSN", "Year"]) #, right_on=["ISSN", "eISSN"])
    
    matched_data = matched_data.reset_index(drop=True)
    return matched_data

In [38]:
# get data
# on full dataset, this takes around 30 seconds
selected_data = get_selected_report_data(type_of_downloads, start_year, start_month, end_year, end_month)
uoft_reports = merge_selected_reports_and_uoft(selected_data)

C:\Users\shirl\Anaconda3\envs\datasci\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\shirl\Anaconda3\envs\datasci\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\shirl\Anaconda3\envs\datasci\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [43]:
uoft_reports

,SP,FilePackage,Year,Journal,Publisher,Platform,Journal DOI,Proprietary Identifier,Print ISSN,Online ISSN,...,b56,a57,b57,a58,b58,a59,b59,a60,b60,NumUofTAuthors
0,No,AAAS,2014,Science,American Association for the Advancement of Sc...,HighWire Press,NaN,NaN,0036-8075,1095-9203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,No,AAAS,2014,Science,American Association for the Advancement of Sc...,HighWire Press,NaN,NaN,0036-8075,1095-9203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,No,AAAS,2014,Science,American Association for the Advancement of Sc...,HighWire Press,NaN,NaN,0036-8075,1095-9203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,No,AAAS,2014,Science,American Association for the Advancement of Sc...,HighWire Press,NaN,NaN,0036-8075,1095-9203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,No,AAAS,2014,Science,American Association for the Advancement of Sc...,HighWire Press,NaN,NaN,0036-8075,1095-9203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90271,No,Wiley Online,2018,PM&R,John Wiley and Sons,Wiley Online Library,10.1002/(ISSN)1934-1563,PMRJ,1934-1482,1934-1563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
90272,No,Wiley Online,2018,PM&R,John Wiley and Sons,Wiley Online Library,10.1002/(ISSN)1934-1563,PMRJ,1934-1482,1934-1563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
90273,No,Wiley Online,2018,PM&R,John Wiley and Sons,Wiley Online Library,10.1002/(ISSN)1934-1563,PMRJ,1934-1482,1934-1563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
90274,No,Wiley Online,2018,PM&R,John Wiley and Sons,Wiley Online Library,10.1002/(ISSN)1934-1563,PMRJ,1934-1482,1934-1563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [42]:
list(uoft_reports.columns)

['SP',
 'FilePackage',
 'Year',
 'Journal',
 'Publisher',
 'Platform',
 'Journal DOI',
 'Proprietary Identifier',
 'Print ISSN',
 'Online ISSN',
 'Reporting Period Total',
 'Reporting Period HTML',
 'Reporting Period PDF',
 'Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec',
 'Downloads',
 'UID',
 'PubDate',
 'Issue',
 'Volume',
 'Pages',
 'Start Page',
 'End Page',
 'Number of Pages',
 'Source Title',
 'Source Title: Abbr',
 'Item Title',
 'Document Type',
 'Normalized Document Type',
 'Publisher: Display Name',
 'Publisher: Full Name',
 'Language',
 'Category: Heading 1',
 'Category: Subheadings',
 'Category: Subjects',
 'Grant Agency',
 'Total number of authors',
 "(a1) First UofT affiliated author's position in the author list ",
 " (b1) First UofT author's name",
 " (a2) Second UofT affiliated author's position in the author list",
 "(b2) Second UofT author's name ",
 "(a3) Third UofT affiliated author's position in the author list ",
 " 